# Logistic Regression

로지스틱회귀는 이진분류문제를 풀기위한 모형이며 종속변수 Y는 범주형 변수로 2가지 값 0또는 1만을 가질 수 있습니다. 파라미터를 사용하여 종속변수 y값을 설명한다는 점에서 선형회귀와 비슷하지만 y가 범주형변수라는 점과 분류문제에 사용되는 모델이라는 차이점이 있습니다.

# 가정
로지스틱회귀의 기본 가정은 다음과 같습니다.

`-` 각각의 데이터요소(샘플)는 여러개의 독립변수와 0또는1만 가질수 있는 종속변수로 이루어져 있습니다. <br>
`-` 임의의 m개의 독립변수와 이진 종속변수를 가지는 i번째 샘플을 나타나면 다음과 같습니다.<br>

$$(x_{1,i},x_{2,i},\dots,x_{m,i}),Y_{i}$$

`-` $x_{1,i},x_{2,i},\dots,x_{m,i}$가 주어진 상황에서 종속변수 $Y_{i}$는 베르누이 분포를 따르는 확률변수입니다.<br>

\begin{aligned}
& Y_i|x_{1,i},x_{2,i},\dots,x_{m,i} \sim \text{Bernouli}(p_{i}) \nonumber \\
& \text{where},p_{i} = P(Y|X=1)
\end{aligned}

`-` 확률변수$Y_{i}$가 따르는 확률함수를 구체적으로 표현한 식입니다.<br>
$$Pr\,(Y_{i} = y|x_{1,i},x_{2,i},\dots,x_{m,i}\,) =
\begin{cases} 
p_i   & \text{if }y=1, \\
1-p_i & \text{if }y=0
\end{cases} = p_i^y(1-p_i)^{1-y}$$
<br>

`-` 기댓값은 실험 또는 시행을 무한히 반복했을때 확률변수가 취하는 값의 평균으로 기대되는(expected) 값(value)입니다. 베르누이분포를 따르는 확률변수 $Y_{i}$의 기댓값을 구하면 다음과 같습니다.<br>

$$E\,[Y_i|x_{1,i},x_{2,i},\dots,x_{m,i}]\, = \sum_{i=0}^{i=1}p_iy_i = 1 \times p_{i} + 0 \times (1-p_{i}) = p_{i}$$


## 기댓값에 대한 고찰
기댓값은 실험 또는 시행을 무한히 반복했을때 확률변수가 취하는 값의 평균으로(또는 샘플링된 값의 평균) 기대되는 값입니다. 확률변수가 베르누이 분포를 따르는 경우 확률변수에 대한 기댓값($E\,[Y_i|x_{1,i},x_{2,i}\.,\dots,x_{m,i}]$)과 모수$(p_i)$가 같은 값을 가집니다. 그러므로,만약에 주어진 샘플데이터로부터 베르누이분포의 모수를 적절히 추정할 수 있다면 주어진 조건하에서 실험 또는 시행을 무한히 반복할 경우 확률변수가 1인사건과 0인사건중 어떤 사건이 더 많이 발생할지 알 수 있고 이를 바탕으로 종속변수 Y의 값을 결정하는 것은 타당합니다.<br>
`-` e.g.<br>

- $E\,[y_i]\, = \hat{p_i}<0.5$ => 무한히 실행했을때 0인 경우가 더 많을 것임 => 관측치를 0으로 예측
<br>
- $E\,[y_i]\, = \hat{p_i}\geq0.5$=>무한히 실행했을때 1인 경우가 더 많을 것임 => 관측치를 1로 예측

# 로지스틱회귀 수식 유도

## concept

선형회귀에서 추정하고자하는 변수$y$는 $x_0,x_1,...,x_m$과 $w_0,w_1,...,w_m$과의 linear combination이였습니다.위에서 언급했듯이 특정샘플에 대한 모수를 적절하게 추정할 수 있다면 관측치가 어떤 클래스에 속할지 합리적으로 알 수 있으므로,로지스틱회귀에서도 선형회귀에서의 아이디어를 핵심아이디어를 가지고와서 추정하고자 하는 모수$p_i$를  $x_0,x_1,...,x_m$과 $w_0,w_1,...,w_m$의 linear combination로 표현하고자 합니다.<br>

`선형회귀의 아이디어(linear combination)` + `모수에 대한 표현`이라는 조건을 만족하기 위해서 최종적인 식은 다음과 조건을 만족해야 것입니다.<br>
`-` $(x_0,x_1,...,x_m)\,,(w_0,w_1,..,w_m)$의 linear combination 식에 있어야 함.<br>
`-` linearcombination = 모수(추정하고자하는값)여야 함.

why linear combination?

## 본격적인 유도

### 모수를 로지스틱함수로 바꾸기

1. $(x_0,x_1,...,x_m)\,,(w_0,w_1,..,w_m)$의 linear combination이 식에 존재해야 합니다. 그러므로 선형방정식을 하나 만듭니다.<br>
\begin{align}
f(i) = x_{1,i}w_0 + x_{2,i}w_1 + x_2w_2 + ... + x_{m,i}w_m = X_iW \nonumber \\
where,X_i = \,[x_{1,i},x_{2,i},\dots,x_{m,i}]\, ,W = \,[w_0,w_1,\dots,w_m]^\text{T} \nonumber \\
\end{align}
2. 좌변은 예측하고자 하는 값인 모수여야 합니다. 좌변을 바꿔봅니다.
$$p_i = WX_i$$
3. 좌변의 베르누이 분포의 모수 $p_i$는 확률변수 $Y_i = 1$인 사건이 일어날 확률입니다. 그러므로 $[0,1]$이라는 범위를 가지는 반면 우변의 값$WX_i$은 $\,[-\infty,\infty]\,$에 범위를 가집니다.  여기서 Odss Ratio를 써서 모수 $p_i$를 포함하며 더 넓은 range를 갖도록 좌변을 수정합니다.
$$\text{Odds Ratio} = \frac{p_i}{1-p_i} = WX_i$$
4. 좌변을 Odds Ratio로 수정했지만 여전히 좌변의 범위는$\,[0,\infty]\,$으로 우변에 비해 좁습니다. 따라서 Odds Ratio에 로짓변환을 취하여 좌변의 범위를 $\,[-\infty,\infty]$로 넓혀줍니다.
$$\text{logit}(p) = \text{ln}\frac{p_i}{1-p_i} = WX_i$$

위 식을 해석하기 위해 $X$의 첫번째 요소인 $x_1$에 대응하는 회귀계수 $w_1$이 학습결과 3으로 정해졌다고 가정해봅시다.만약 $x_1$의 값이 1증가한다면 로그오즈비가 3증가합니다.<br>

5. 이제 양변의 범위는 맞춰졌으므로 추정하고자 하는 변수 $p_i$가 좌변에 오도록 정리해봅시다.
$$p_i = \frac{1}{\,(1 + e^{-WX_i})\, }$$
(전개)<br>
$\frac{p_i}{1-p_i} = e^{WX_i}$<br>
$\Longleftrightarrow p_i = \,(1-p_i)\,e^{WX_i}$<br>
$\Longleftrightarrow p_i = \,e^{WX_i}-p_ie^{WX_i}$<br>
$\Longleftrightarrow p_i + p_ie^{WX_i} = \,e^{WX_i}$<br>
$\Longleftrightarrow p_i\,(1 + e^{WX_i})\, = \,e^{WX_i}$<br>
$\Longleftrightarrow p_i = \frac{\,e^{WX_i}}{\,(1 + e^{WX_i})\, }$<br>
$\Longleftrightarrow p_i = \frac{1}{\,(1 + e^{-WX_i})\, }$<br>

최종적으로, 앞서 목적이었던 X와 W의 선형조합이 수식내부에 존재하도록 새롭게 표현한 모수는 다음과 같습니다.
$$p_i(y_i) = Pr\,(Y_i = 1|X_i;W)\, = \frac{1}{\,1 + e^{-WX_i}\,}$$

### 베르누이 분포의 pmf 정리

베르누이분포의 모수 $p_i$가 새롭게 표현되었으므로 확률질량함수도 새롭게 표현할 수 있습니다.<br>

\begin{align}
Bern(y_i;p) = Pr\,(Y_{i} = y_i|x_{1,i},x_{2,i},\dots,x_{m,i};p_i) &= 
\begin{cases} 
p_i & \text{if}\,y_i=1 \\
1-p_i & \text{if}\,y_i=0 
\end{cases} \\
&= p_i^{y_i}(1-p_i)^{1-y_i} \\ 
&= \frac{e^{y_iWX_i}}{1+e^{WX_i}}
\end{align}
<br>

참고 : (3번식 전개)<br>
\begin{aligned}
p_i^y(1-p_i)^{1-y} &= \ (\frac{1}{\,1 + e^{-WX_i}\,})^y\,\,(1-\frac{1}{\,1 + e^{-WX_i}\,})^{1-y} \\
&= (\frac{1}{1+e^{-WX_i}})^{y_i}(\frac{e^{-WX_i}}{1+e^{-WXi}})^{1-y_i} \\
&= (\frac{1}{1+e^{-WX_i}})^{y_i}(\frac{1}{1+e^{WXi}})^{1-y_i} \\
&= (\frac{1+e^{WX_i}}{1+e^{-WX_i}})^{y_i}(\frac{1}{1+e^{WX_i}}) \\
&= (\frac{e^{WX_i}+e^{2WX_i}}{1+e^{WX_i}})^{y_i}(\frac{1}{1+e^{WX_i}}) \\
&= (\frac{e^{WX_i}(1+e^{WX_i})}{1+e^{WX_i}})^{y_i}(\frac{1}{1+e^{WX_i}}) \\
&= e^{y_iWX_i}\frac{1}{1+e^{WX_i}} \\
&= \frac{e^{y_iWX_i}}{1+e^{WX_i}}
\end{aligned}



# MLE 

f